# 5.2 線形判別分析(LDA)による教師ありデータ圧縮
- LDAは、計算効率を高め、正則化されていないモデルで「次元の呪い」による過学習を抑制するために使用できる。
- LDAはクラスの分離を最適化する特徴部分空間を見つけ出そうとする。
- LDAは教師ありアルゴリズムであるため、分類タスクの特徴抽出の手段としては、LDAの方が優れている。
- LDAではデータが正規分布に従っていることが前提である。
- LDAではクラスの共分散行列がまったく同じであることが前提である。
- LDAでは特徴量が統計的に見て互いに独立していることも前提となる。
- 以上の3つの前提を多少満たしていなくとも、次元削減の手段としてのLDAはそれなりにうまく行く
- LDAのアプローチの主な手順についてまとめる

１．d次元のデータセットを標準化する

２．クラスごとにd次元の平均ベクトルを計算する

３．クラス間変動行列$\vec{S}_{B}$と、クラス内変動行列$\vec{S}_{W}$を生成する

４．行列$\vec{S}^{-1}_{W} \vec{S}_{B}$の固有ベクトルと対応する固有値を計算する

５．$d\times k$次元の変換行列$\vec{W}$を生成するために、最も大きいk個の固有値に対応するk個の固有ベクトルを選択する。固有ベクトルは、この行列の列である。

６．変換行列$\vec{W}$を使ってサンプルを新しい特徴部分空間へ射影する。

# 5.2.1 変動行列を計算する
- 平均ベクトルを計算する。
- 各平均ベクトル$\vec{m}_{i}$には、クラス$i$のサンプルに関する平均特徴量の値$\mu_{m}$が格納される。
$$\vec{m}_{i} = \frac{1}{n_{i}} \sum_{\vec{x} \in D_{i}} \vec{x}$$
- 平均ベクトルを用いて、クラス内変動行列とクラス間変動行列を生成する。

- wineデータセットの例では、以下の平均ベクトルが3つ得られる。
$$\vec{m}_{i} = \begin{bmatrix} \mu_{i,alcohol} \\ \mu_{i,malic acid} \\ \vdots \\ \mu_{i,proline} \end{bmatrix} \, i\in (1,2,3)$$ 

In [2]:
import pandas as pd

df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',header=None)

from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler

X,y = df_wine.iloc[:,1:].values ,df_wine.iloc[:,0].values
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [3]:
import numpy as np

np.set_printoptions(precision=4)
mean_vecs = []

for label in range(1,4):
    mean_vecs.append(np.mean(X_train_std[y_train==label],axis=0))
    print('MV %s: %s\n') %(label,mean_vecs[label-1])

MV 1: [ 0.9259 -0.3091  0.2592 -0.7989  0.3039  0.9608  1.0515 -0.6306  0.5354
  0.2209  0.4855  0.798   1.2017]

MV 2: [-0.8727 -0.3854 -0.4437  0.2481 -0.2409 -0.1059  0.0187 -0.0164  0.1095
 -0.8796  0.4392  0.2776 -0.7016]

MV 3: [ 0.1637  0.8929  0.3249  0.5658 -0.01   -0.9499 -1.228   0.7436 -0.7652
  0.979  -1.1698 -1.3007 -0.3912]



- 平均ベクトルを使ってクラス内変動行列$S_{W}$を計算する方法は以下のようになる
$$\vec{S}_{W} = \sum^{c}_{i=1} \vec{S}_{i}$$
$$\vec{S}_{i} = \sum_{\vec{x} \in D_{i}}(\vec{x} - \vec{m}_{i})(\vec{x}-\vec{m}_{i})^{T}$$

In [22]:
d = 13 #特徴量の個数
S_W = np.zeros((d,d))
for label,mv in zip(range(1,4),mean_vecs):
    class_scatter = np.zeros((d,d))
    for row in X_train_std[y_train == label]:
        row,mv = row.reshape(d,1),mv.reshape(d,1) #list of listに変換
        class_scatter += (row-mv).dot((row-mv).T)
    S_W += class_scatter

print('within-class scatter matrix: %sx%s') % (S_W.shape[0],S_W.shape[1])

within-class scatter matrix: 13x13


- 変動行列を計算するときには、トレーニングデータセットにおいてクラスラベルが一様に分布していることが前提となる。このデータセットでは、この前提を満たしていない。

In [32]:
print('Class label distribution: %s') %np.bincount(y_train)[1:]

Class label distribution: [40 49 35]


- 個々の変動行列$\vec{S}_{i}$を合計して変動行列$\vec{S}_{W}$を生成する前に、スケーリングが必要となる。
- 変動行列をクラスごとのサンプルの個数$N_{i}$で割るときに、変動行列の計算が実は共分散行列の計算と同じであることがわかる。
- 共分散行列は変動行列の正規化バージョンである。
$$\sum_{i} = \frac{1}{N_{i}}\vec{S}_{W} = \frac{1}{N_{i}}\sum_{\vec{x} in D_{i}} (\vec{x}-\vec{m}_{i})(\vec{x}-\vec{m}_{i})^{T}$$

In [37]:
d = 13 #特徴量の個数
S_W = np.zeros((d,d))
for label,mv in zip(range(1,4),mean_vecs):
    class_scatter = np.cov(X_train_std[y_train==label].T)
    S_W += class_scatter
    
print('Scaled within-class scatter matrix: %sx%s') % (S_W.shape[0],S_W.shape[1])
    

Scaled within-class scatter matrix: 13x13


- クラス間変動行列$S_{B}$の計算
- $\vec{m}$はすべてのクラスのサンプルを対象として計算される全体平均である
$$\vec{S}_{B} = \sum^{c}_{i=1} N_{i}(\vec{m}_{i}-\vec{m})(\vec{m}_{i}-\vec{m})^{T}$$

In [39]:
mean_overall = np.mean(X_train_std,axis=0)
d = 13 # 特徴量の個数
S_B = np.zeros((d,d))
for i,mean_vec in enumerate(mean_vecs):
    n = X_train[y_train==i+1,:].shape[0]
    mean_vec = mean_vec.reshape(d,1)
    mean_overall = mean_overall.reshap(d,1)
    S_B += n * (mean_vec - mean_overall).dot((mean_vec - mean_overall).T)

print('Between-class scatter matrix: %sx%s') %(S_B.shape[0],S_B.shape[1])

AttributeError: 'numpy.ndarray' object has no attribute 'reshap'